#**Aula 2: Categorizando produtos**

##**Categorizando um produto**

Modelo: [bart-large-mnli](https://huggingface.co/facebook/bart-large-mnli)

In [1]:
from transformers import pipeline
import torch

device = torch.device("mps") if torch.backends.mps.is_built() else torch.device("cpu")
print("Dispositivo:", device)

Dispositivo: mps


In [2]:
classificador = pipeline('zero-shot-classification', model='facebook/bart-large-mnli', device=device)

In [3]:
categorias = ['eletronics', 'food', 'toys', 'books']

In [4]:
predicao = classificador('''
Latest model of smartphone with 5G connectivity and 128GB internal storage''', candidate_labels=categorias)

In [5]:
predicao

{'sequence': '\nLatest model of smartphone with 5G connectivity and 128GB internal storage',
 'labels': ['eletronics', 'food', 'books', 'toys'],
 'scores': [0.9009812474250793,
  0.037431515753269196,
  0.03276659920811653,
  0.028820620849728584]}

##**Selecionando o modelo**

Modelo: [comprehend-it-multilang-base](https://huggingface.co/knowledgator/comprehend_it-multilingual-t5-base)

In [ ]:
# pip install liqfit sentencepiece

In [6]:
from liqfit.pipeline import ZeroShotClassificationPipeline
from liqfit.models import T5ForZeroShotClassification
from transformers import T5Tokenizer

model = T5ForZeroShotClassification.from_pretrained('knowledgator/comprehend_it-multilingual-t5-base')
tokenizer = T5Tokenizer.from_pretrained('knowledgator/comprehend_it-multilingual-t5-base')
classifier = ZeroShotClassificationPipeline(model=model, tokenizer=tokenizer,
                                                      hypothesis_template = '{}', encoder_decoder = True)

You are using a model of type T5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [7]:
descricao = '''Este water gel leve e refrescante, proporciona hidratação imediata que ajuda a aliviar o
repuxamento e aspereza da pele sensível'''
categorias_candidatas = ['beleza', 'cozinha', 'livros']
resultado = classifier(descricao, categorias_candidatas, multi_label=False, device=device)
resultado

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'sequence': 'Este water gel leve e refrescante, proporciona hidratação imediata que ajuda a aliviar o\nrepuxamento e aspereza da pele sensível',
 'labels': ['beleza', 'livros', 'cozinha'],
 'scores': [0.9420966506004333, 0.03873449191451073, 0.019168870523571968]}

In [8]:
import pandas as pd

resultado = pd.DataFrame(resultado).drop(['sequence'], axis=1)
resultado

,labels,scores
0,beleza,0.942097
1,livros,0.038734
2,cozinha,0.019169


In [9]:
descricao = '''
A fritadeira eletrica sem óleo start fry da elgin possui um design unico, capacidade para até 3,5 litros,
potência de 1400w e revestimento antiaderente. Seu sistema de circulação de ar ultrarapido frita e economiza
energia. Sua grelha de fritura é removivel e super fácil de limpar. Ela conta com uma proteção contra
super aquecimento. Possui controle de temperatura de 80°c a 200°c que permite você programar a temperatura
de preparo para cada tipo de alimento,timer para até 60 minutos com aviso sonoro e desligamento automático,
assim você pode deixar preparando sua refeição enquanto realiza outras tarefas.
'''
categorias_candidatas = ['beleza', 'cozinha', 'livros']
resultado = classifier(descricao, categorias_candidatas, multi_label=False, device=device)
resultado = pd.DataFrame(resultado).drop(['sequence'], axis=1)
resultado

,labels,scores
0,cozinha,0.718628
1,beleza,0.235084
2,livros,0.046288


##**Aplicando o modelo aos dados**

In [10]:
def visualizar_dataframe(df):
  """
  Creates a styled visualization of a DataFrame with left-aligned text.
  
  Args:
    df (pandas.DataFrame): The DataFrame to style
    
  Returns:
    pandas.io.formats.style.Styler: A styled DataFrame
  """
  visualizacao = (
    df.style
    .set_properties(**{'text-align': 'left'})
    .set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])
  )
  
  return visualizacao

In [11]:
dados = pd.read_csv('../Dados/descricoes_produtos.csv')
visualizar_dataframe(dados)

,Descrição
0,Liquidificador de alta potência com jarra de vidro e várias velocidades.
1,"Forno Micro-ondas de 20 litros, com menu descongelar."
2,Máquina de café espresso com reservatório de água e moedor integrado.
3,Torradeira com capacidade para quatro fatias e controle de temperatura.
4,"Panela elétrica multifuncional que cozinha, assa e frita sem óleo."
5,Smartphone com 128GB de armazenamento e câmera de 12MP.
6,"Smartwatch monitoramento de passos, frequência cardíaca e bateria de longa duração."
7,Notebook gamer com placa de vídeo potente e tela de alta resolução.
8,"Smartphone com tela super retina XDR de 6,7 polegadas, câmera tripla avançada com tecnologia ProRAW e desempenho incomparável graças ao chip A16 Bionic."
9,Câmera digital com zoom óptico de 20x e vídeos em alta definição.


In [12]:
categorias = ['eletrodomésticos', 'eletrônicos', 'beleza', 'brinquedos']

def categorizar(descricao):
  resultado = classifier(descricao, categorias, multi_label=False, device=device)
  categoria_max = max(zip(resultado['labels'],resultado['scores']), key=lambda x: x[1])[0]
  return categoria_max

dados['Categoria'] = dados['Descrição'].apply(categorizar)

In [13]:
visualizar_dataframe(dados)

,Descrição,Categoria
0,Liquidificador de alta potência com jarra de vidro e várias velocidades.,eletrodomésticos
1,"Forno Micro-ondas de 20 litros, com menu descongelar.",eletrodomésticos
2,Máquina de café espresso com reservatório de água e moedor integrado.,eletrodomésticos
3,Torradeira com capacidade para quatro fatias e controle de temperatura.,eletrodomésticos
4,"Panela elétrica multifuncional que cozinha, assa e frita sem óleo.",eletrodomésticos
5,Smartphone com 128GB de armazenamento e câmera de 12MP.,eletrônicos
6,"Smartwatch monitoramento de passos, frequência cardíaca e bateria de longa duração.",eletrônicos
7,Notebook gamer com placa de vídeo potente e tela de alta resolução.,eletrônicos
8,"Smartphone com tela super retina XDR de 6,7 polegadas, câmera tripla avançada com tecnologia ProRAW e desempenho incomparável graças ao chip A16 Bionic.",eletrônicos
9,Câmera digital com zoom óptico de 20x e vídeos em alta definição.,eletrônicos


# Atividade 8

In [ ]:
# pip install liqfit sentencepiece

In [14]:
from liqfit.pipeline import ZeroShotClassificationPipeline
from liqfit.models import T5ForZeroShotClassification
from transformers import T5Tokenizer

model = T5ForZeroShotClassification.from_pretrained('knowledgator/comprehend_it-multilingual-t5-base')
tokenizer = T5Tokenizer.from_pretrained('knowledgator/comprehend_it-multilingual-t5-base')
classifier = ZeroShotClassificationPipeline(model=model, tokenizer=tokenizer,
                                                      hypothesis_template = '{}', encoder_decoder = True, device=device)

You are using a model of type T5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


In [15]:
dados = pd.read_csv('../Dados/produtos_descricoes.csv')
visualizar_dataframe(dados)

,Descrição
0,"Carrinho de brinquedo para crianças com estrutura resistente e cores vibrantes, ideal para crianças pequenas se divertirem e melhorarem a coordenação motora."
1,"Boneca com cabelo macio, vestuário intercambiável e vários acessórios para um jogo criativo e divertido."
2,"Conjunto de blocos de construção em diferentes formas e cores, perfeito para estimular a criatividade e a habilidade de construção das crianças."
3,"Quebra-cabeça com uma imagem deslumbrante, projetado para desafiar e entreter crianças e adultos."
4,"Trenzinho elétrico com trilhos e vagões, movido a bateria, que oferece diversão interativa e educativa."
5,"Brinquedo interativo que ajuda no desenvolvimento cognitivo e motor das crianças, com atividades educativas e lúdicas."
6,"Conjunto de roupas de banho coloridas e confortáveis para crianças, com estampas divertidas e tecido rápido de secar."
7,"Carro de controle remoto com alta velocidade e direção precisa, ideal para brincar em casa ou ao ar livre."
8,"Kit completo com tintas, pincéis e papel para crianças explorarem sua criatividade e habilidades artísticas."
9,"Boneca interativa que fala e canta, com diferentes frases e músicas, proporcionando uma experiência divertida e educativa."


In [16]:
categorias = ['brinquedos', 'beleza', 'eletrônicos', 'ferramentas' ,'cozinha']

def categorizar(descricao):
  resultado = classifier(descricao, categorias, multi_label=False, device=device)
  categoria_max = max(zip(resultado['labels'],resultado['scores']), key=lambda x: x[1])[0]
  return categoria_max

dados['Categoria'] = dados['Descrição'].apply(categorizar)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [17]:
visualizar_dataframe(dados)

,Descrição,Categoria
0,"Carrinho de brinquedo para crianças com estrutura resistente e cores vibrantes, ideal para crianças pequenas se divertirem e melhorarem a coordenação motora.",brinquedos
1,"Boneca com cabelo macio, vestuário intercambiável e vários acessórios para um jogo criativo e divertido.",brinquedos
2,"Conjunto de blocos de construção em diferentes formas e cores, perfeito para estimular a criatividade e a habilidade de construção das crianças.",brinquedos
3,"Quebra-cabeça com uma imagem deslumbrante, projetado para desafiar e entreter crianças e adultos.",brinquedos
4,"Trenzinho elétrico com trilhos e vagões, movido a bateria, que oferece diversão interativa e educativa.",brinquedos
5,"Brinquedo interativo que ajuda no desenvolvimento cognitivo e motor das crianças, com atividades educativas e lúdicas.",brinquedos
6,"Conjunto de roupas de banho coloridas e confortáveis para crianças, com estampas divertidas e tecido rápido de secar.",brinquedos
7,"Carro de controle remoto com alta velocidade e direção precisa, ideal para brincar em casa ou ao ar livre.",brinquedos
8,"Kit completo com tintas, pincéis e papel para crianças explorarem sua criatividade e habilidades artísticas.",brinquedos
9,"Boneca interativa que fala e canta, com diferentes frases e músicas, proporcionando uma experiência divertida e educativa.",brinquedos
